# LeTCI Results Analysis

This notebook demonstrates LeTCI (Level-Trend-Comparison-Integration) results scoring with trend visualization.

## Learning Objectives
1. Compute LeTCI scores from results data
2. Analyze Level-Trend-Comparison-Integration dimensions
3. Visualize performance trends over time
4. Compare results against benchmarks

In [ ]:
import sys
sys.path.append('../src')

from adli_letci_core import LeTCIIndicators, compute_letci_score

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (12, 6)

print("Libraries loaded successfully!")

## Load Results Data

In [ ]:
# Load sample data
df = pd.read_csv('../data/examples/sample_assessment_data.csv')

# Filter results items
results_df = df[df['item_type'] == 'Results'].copy()

print(f"Loaded {len(results_df)} results assessments")
print(f"Departments: {results_df['department'].unique()}")

results_df.head()

## LeTCI Scoring Example

In [ ]:
# Example: Score Results Item 7.1 (Student Learning Outcomes)
item = results_df[(results_df['item_id'] == '7.1') & (results_df['department'] == 'Computer Science')].iloc[0]

letci = LeTCIIndicators(
    level=item['level'],
    trend=item['trend'],
    comparison=item['comparison'],
    integration=item['integration']
)

score = compute_letci_score(letci)

print(f"Results Item 7.1 - Computer Science")
print(f"\nLeTCI Dimensions:")
print(f"  Level:       {letci.level:.3f} (40% weight) = {letci.level * 0.40:.3f}")
print(f"  Trend:       {letci.trend:.3f} (25% weight) = {letci.trend * 0.25:.3f}")
print(f"  Comparison:  {letci.comparison:.3f} (25% weight) = {letci.comparison * 0.25:.3f}")
print(f"  Integration: {letci.integration:.3f} (10% weight) = {letci.integration * 0.10:.3f}")
print(f"\nLeTCI Score: {score:.2f}/100")

## LeTCI Dimensional Breakdown

In [ ]:
# Compute scores for all results items
results_df['letci_score'] = results_df.apply(
    lambda row: compute_letci_score(
        LeTCIIndicators(row['level'], row['trend'], row['comparison'], row['integration'])
    ),
    axis=1
)

# Department-level means
dept_letci = results_df.groupby('department')[['level', 'trend', 'comparison', 'integration', 'letci_score']].mean()

# Visualize
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

dimensions = ['level', 'trend', 'comparison', 'integration']
titles = ['Level (Current Performance)', 'Trend (Improvement Rate)', 
          'Comparison (vs. Benchmarks)', 'Integration (Alignment)']
colors = ['#2ca02c', '#1f77b4', '#ff7f0e', '#9467bd']

for idx, (dim, title, color) in enumerate(zip(dimensions, titles, colors)):
    ax = axes[idx // 2, idx % 2]
    dept_letci[dim].plot(kind='barh', ax=ax, color=color)
    ax.set_xlabel('Score', fontsize=11)
    ax.set_ylabel('Department', fontsize=11)
    ax.set_title(title, fontsize=12, fontweight='bold')
    ax.set_xlim(0, 1.0)
    ax.grid(axis='x', alpha=0.3)

plt.suptitle('LeTCI Dimensional Analysis by Department', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()

print("\nDepartment LeTCI Scores:")
print(dept_letci)

## Time-Series Trend Visualization

In [ ]:
# Load benchmark data with time series
benchmark_df = pd.read_csv('../data/examples/benchmark_results.csv')

# Plot LeTCI components over time
fig, axes = plt.subplots(2, 2, figsize=(15, 10))

departments = ['Computer Science', 'Business Admin', 'Engineering']
colors_dept = {'Computer Science': '#1f77b4', 'Business Admin': '#ff7f0e', 'Engineering': '#2ca02c'}

letci_cols = ['mean_letci_level', 'mean_letci_trend', 'mean_letci_comparison']
titles_ts = ['Level Over Time', 'Trend Over Time', 'Comparison Over Time', 'Overall Results Score']

for idx, (col, title) in enumerate(zip(letci_cols + ['organizational_score'], titles_ts)):
    ax = axes[idx // 2, idx % 2]
    
    for dept in departments:
        dept_data = benchmark_df[benchmark_df['department'] == dept]
        ax.plot(dept_data['month'], dept_data[col], marker='o', 
               linewidth=2, label=dept, color=colors_dept[dept])
    
    ax.set_xlabel('Month', fontsize=11)
    ax.set_ylabel('Score', fontsize=11)
    ax.set_title(title, fontsize=12, fontweight='bold')
    ax.legend(fontsize=10)
    ax.grid(alpha=0.3)
    ax.set_xlim(1, 12)

plt.suptitle('12-Month LeTCI Performance Trends', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()

## Performance Growth Analysis

In [ ]:
# Compute growth rates
growth_analysis = []

for dept in departments:
    dept_data = benchmark_df[benchmark_df['department'] == dept]
    initial = dept_data.iloc[0]['organizational_score']
    final = dept_data.iloc[-1]['organizational_score']
    growth = final - initial
    growth_pct = (growth / initial) * 100
    
    growth_analysis.append({
        'Department': dept,
        'Initial': initial,
        'Final': final,
        'Growth': growth,
        'Growth %': growth_pct
    })

growth_df = pd.DataFrame(growth_analysis)

# Visualize growth
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

# Absolute growth
ax1.barh(growth_df['Department'], growth_df['Growth'], color=['#1f77b4', '#ff7f0e', '#2ca02c'])
ax1.set_xlabel('Score Improvement (points)', fontsize=12)
ax1.set_ylabel('Department', fontsize=12)
ax1.set_title('Absolute Performance Growth\n(12-Month Period)', fontsize=13, fontweight='bold')
ax1.grid(axis='x', alpha=0.3)

# Percentage growth
ax2.barh(growth_df['Department'], growth_df['Growth %'], color=['#1f77b4', '#ff7f0e', '#2ca02c'])
ax2.set_xlabel('Growth Rate (%)', fontsize=12)
ax2.set_ylabel('Department', fontsize=12)
ax2.set_title('Relative Performance Growth\n(12-Month Period)', fontsize=13, fontweight='bold')
ax2.grid(axis='x', alpha=0.3)

plt.tight_layout()
plt.show()

print("\n12-Month Growth Analysis:")
print(growth_df.to_string(index=False))

## LeTCI vs ADLI Correlation

In [ ]:
# Analyze correlation between process (ADLI) and results (LeTCI)
fig, ax = plt.subplots(figsize=(10, 6))

for dept in departments:
    dept_data = benchmark_df[benchmark_df['department'] == dept]
    
    # Use mean ADLI approach as proxy for process maturity
    process_score = (dept_data['mean_adli_approach'] + dept_data['mean_adli_deployment']) / 2
    results_score = dept_data['mean_letci_level']
    
    ax.scatter(process_score, results_score, s=100, alpha=0.6, 
              color=colors_dept[dept], label=dept, edgecolors='black')
    
    # Fit trend line
    z = np.polyfit(process_score, results_score, 1)
    p = np.poly1d(z)
    ax.plot(process_score, p(process_score), linestyle='--', 
           color=colors_dept[dept], alpha=0.5, linewidth=2)

ax.set_xlabel('Mean Process Score (ADLI)', fontsize=12)
ax.set_ylabel('Mean Results Level (LeTCI)', fontsize=12)
ax.set_title('Process-Results Correlation Analysis', fontsize=14, fontweight='bold')
ax.legend(fontsize=11)
ax.grid(alpha=0.3)

plt.tight_layout()
plt.show()

print("\nKey Insight: Strong ADLI process scores correlate with high LeTCI results levels,")
print("confirming that systematic processes drive performance outcomes.")

## Summary

### LeTCI Analysis Insights:
1. **Level Emphasis**: 40% weight on current performance ensures focus on actual outcomes
2. **Trend Importance**: 25% weight rewards sustained improvement trajectories
3. **Benchmark Context**: 25% weight on comparisons drives competitive excellence
4. **Integration Link**: 10% weight ensures results align with strategic goals

### Typical Findings:
- **High Level, Low Trend**: Good current performance but plateauing
- **Strong Trend, Weak Comparison**: Improving but still below benchmarks
- **Low Integration**: Results not aligned with organizational priorities

## Next Steps
- **Notebook 04**: Complete organizational assessment
- **Notebook 06**: Integration Health Index trajectory analysis